In [1]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('data/train.csv')

to_delete_var = ['qa_id', 'url',
                 'question_user_name', 'question_user_page',
                 'answer_user_name', 'answer_user_page']

# Exploration empirique des données selon les valeurs des targets
i=11

data = train.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,i]].drop(to_delete_var, 1)
data.sort_values(by=train.columns[i]).head()

,question_title,question_body,answer,category,host,question_asker_intent_understanding
4228,Tuples in LaTeX,I am learning to use TikZ to draw things in La...,You can use the xparse package:\n\n\documentcl...,TECHNOLOGY,tex.stackexchange.com,0.333333
914,How to select multiple non-sequential lines in...,Is there any way to select multiple lines in V...,The multiselect - Create multiple selections a...,TECHNOLOGY,superuser.com,0.333333
5993,Group isomorphisms and a possible trivial stat...,"I have the following set $G=\lbrace a,b,e \rbr...",Every group of three elements is isomorphic to...,SCIENCE,math.stackexchange.com,0.333333
1522,"Deploying Yesod to Heroku, can't build statically",I'm very new to Yesod and I'm having trouble b...,You have several issues.\n\nYou should not bui...,STACKOVERFLOW,stackoverflow.com,0.333333
5039,Should you optimize mobile experiences based o...,"In abbood's question ""How to implement a stack...",(EDIT: The question was clarified a bit since ...,TECHNOLOGY,ux.stackexchange.com,0.333333


In [18]:
X_train = train.iloc[:, :11].drop(to_delete_var, 1)
y_train = train.iloc[:, 11:]

y_train_transformed = y_train.apply(lambda x: pd.cut(x,
                                                     [-0.1, .25, .5, .75, 1.1],
                                                     labels=['low', 'medium-', 'medium+', 'high']))

In [4]:
vectorizer = CountVectorizer(stop_words='english',
                             lowercase=True,
                             ngram_range=(1, 2))
ohe = OneHotEncoder(drop='first', sparse=True)

text_var = ['question_title', 'question_body', 'answer']

X_train_text = sp.hstack(X_train[text_var].apply(lambda col: vectorizer.fit_transform(col)))
X_train_category = ohe.fit_transform(X_train[['category', 'host']])
X_train_transformed = sp.hstack([X_train_text, X_train_category])

In [42]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def stop_words_filtering(liste):
    for item in liste:
        if item in stop_words:
            liste.remove(item)
            
X = X_train['question_body'].apply(lambda x: word_tokenize(x, language = 'english'))            

In [43]:
X_train.columns
X

0       [After, playing, around, with, macro, photogra...
1       [I, am, trying, to, understand, what, kinds, o...
2       [I, 'm, working, on, a, PCB, that, has, throug...
3       [An, affidavit, ,, from, what, i, understand, ...
4       [I, am, trying, to, make, a, binary, image, .,...
                              ...                        
6074    [I, am, curious, if, anyone, uses, a, skiing, ...
6075    [I, have, a, road, bike, with, a, front, brake...
6076    [I, 'm, tailing, a, log, file, using, tail, -f...
6077    [What, are, people, 's, views, on, this, ?, To...
6078    [Newbie, question, ., Why, is, it, that, there...
Name: question_body, Length: 6079, dtype: object

In [ ]:
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train_transformed,
                                                                            y_train_transformed,
                                                                            test_size=0.15)

dtc = DecisionTreeClassifier(max_depth=5)
dtc_multi = MultiOutputClassifier(dtc, n_jobs=-1)

dtc_multi.fit(X_train_train, y_train_train)
dtc_multi.score(X_train_test, y_train_test)

In [28]:
list(train.iloc[:, 11:].columns)

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [29]:
scores=[]
vectorizer = CountVectorizer(stop_words='english',lowercase=True,ngram_range=(1, 2))
ohe = OneHotEncoder(drop='first', sparse=True)
text_var = ['question_title', 'question_body', 'answer']

for i in range(11, 41):
    y_train_1 = pd.DataFrame(train.iloc[:, i])
    y_train_transformed_1 = y_train_1.apply(lambda x: pd.cut(x,[-0.1, .25, .5, .75, 1.1],
                                                             labels=['low', 'medium-', 'medium+', 'high']))
    X_train_text = sp.hstack(X_train[text_var].apply(lambda col: vectorizer.fit_transform(col)))
    X_train_category = ohe.fit_transform(X_train[['category', 'host']])
    X_train_transformed = sp.hstack([X_train_text, X_train_category])
    X_train_train, X_train_test, y_train_train_1, y_train_test_1 = train_test_split(X_train_transformed,
                                                                                    y_train_transformed_1,
                                                                                    test_size=0.15)
    dtc = DecisionTreeClassifier(max_depth=5)
    dtc.fit(X_train_train, y_train_train_1)
    scores.append(dtc.score(X_train_test, y_train_test_1))
    
scores

[0.8760964912280702,
 0.44298245614035087,
 0.9002192982456141,
 0.4967105263157895,
 0.5789473684210527,
 0.6754385964912281,
 0.5844298245614035,
 0.6293859649122807,
 0.5888157894736842,
 0.9890350877192983,
 0.33881578947368424,
 0.5394736842105263,
 0.9298245614035088,
 0.9791666666666666,
 0.9517543859649122,
 0.881578947368421,
 0.5482456140350878,
 0.6502192982456141,
 0.44956140350877194,
 0.9978070175438597,
 0.7094298245614035,
 0.918859649122807,
 0.7675438596491229,
 0.9583333333333334,
 0.9780701754385965,
 0.8212719298245614,
 0.5460526315789473,
 0.6995614035087719,
 0.40460526315789475,
 0.9331140350877193]

In [31]:
pd.DataFrame({'Target' : list(train.iloc[:, 11:].columns), 'Score' : scores})

,Target,Score
0,question_asker_intent_understanding,0.876096
1,question_body_critical,0.442982
2,question_conversational,0.900219
3,question_expect_short_answer,0.496711
4,question_fact_seeking,0.578947
5,question_has_commonly_accepted_answer,0.675439
6,question_interestingness_others,0.584430
7,question_interestingness_self,0.629386
8,question_multi_intent,0.588816
9,question_not_really_a_question,0.989035
